# Ансамбли

**Автор**: Винник Екатерина Петровна, 22.М04

## Задача

- [ ] Найти данные размера хотя бы 10000, на которых можно решить задачу классификации или регрессии.
- [ ] Выполнить разведочный анализ (**EDA**). При необходимости выполнить полезные преобразования данных.
- [ ] Самостоятельно реализуйте алгоритмы **Bagging** и **AdaBoost**, которые позволят обучить ансамбли каких-то простых моделей, например деревьев принятия решений.
- [ ] Обучите ансамбли для каких-то простых моделей, используя свои две реализации, а также реализации из **sklearn** (**Bagging**, **Random Forest**, **AdaBoost**, **Gradient Tree Boosting**, **Histogram-Based Gradient Boosting**).
- [ ] При обучении использовать **подбор гиперпараметров**, **кросс-валидацию**.
- [ ] Сравнить качество моделей: замерить скорость обучения, вычислить различные метрики качества, построить матрицы ошибок.
- [ ] * (**+2 балла**) Также добавить в сравнение модели из библиотек **XGBoost**, **LightGBM** и **CatBoost**.
- [ ] * (**+3 балла**) Для подбора гиперпараметров использовать фреймворк **Optuna**.

## EDA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import warnings
#warnings.filterwarnings('ignore')  # отключаем предупреждения
df = pd.read_csv('./train.csv')
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income_>50K
0,67,Private,366425,Doctorate,16,Divorced,Exec-managerial,Not-in-family,White,Male,99999,0,60,United-States,1
1,17,Private,244602,12th,8,Never-married,Other-service,Own-child,White,Male,0,0,15,United-States,0
2,31,Private,174201,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
3,58,State-gov,110199,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,0
4,25,State-gov,149248,Some-college,10,Never-married,Other-service,Not-in-family,Black,Male,0,0,40,United-States,0


## Bagging

In [2]:
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier

In [18]:
class BaggedDecisonTreeClassifier:
    
    def __init__(self,num_of_bagged=5):
        # Initialised with number of bagged models
        self.num_of_bagged=num_of_bagged
        
    def fit(self,X,y):
        # to store the models
        self.models=[]
        for i in range(self.num_of_bagged):
            indexs=np.random.choice(len(X),size=len(X))# sample with replacement
            Xi=X[indexs]# Chossing random samples
            Yi=y[indexs]
            # Training for each sample bunch by Decision Tree Classifier
            model=DecisionTreeClassifier()
            model.fit(Xi,Yi)
            # Storing the models
            self.models.append(model)
            
    def predict(self,X):
        pred=np.zeros(len(X))
        # predicting with each stored models
        for model in self.models:
            pred=pred+model.predict(X)
        return np.round(pred/self.num_of_bagged) # Model averaging
    
    def acc(self,y_true,y_pred):
        return np.mean(y_true==y_pred)

In [13]:
df.shape

(43957, 15)

In [14]:
from sklearn.model_selection import GridSearchCV, train_test_split

y = df["income_>50K"]
X = df.drop(["income_>50K"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=55)

bdtc=BaggedDecisonTreeClassifier(10)
bdtc.fit(X_train,y_train)

ValueError: could not convert string to float: 'Private'

In [19]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X=data.data
y=data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
bdtc=BaggedDecisonTreeClassifier(10)
bdtc.fit(X_train,y_train)
y_pred=bdtc.predict(X_test)
bdtc.acc(y_test,y_pred)

0.9298245614035088

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, y_train)
y_pred_clf=clf.predict(X_test)
accuracy_score(y_test, y_pred_clf)


0.9415204678362573

In [ ]:
y_pred=bdtc.predict(X_test)

In [ ]:
bdtc.acc(y_test,y_pred)

## AdaBoost

## Обучение моделей